In [17]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
data=pd.read_csv('Reviews.csv')
data.shape

(568454, 10)

In [19]:
data.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [20]:
processed_data=data.dropna()

In [21]:
processed_data.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [22]:
new_data=processed_data.head(30000)
new_data['Score']

0        5
1        1
2        4
3        2
4        5
        ..
29997    5
29998    5
29999    5
30000    5
30001    5
Name: Score, Length: 30000, dtype: int64

In [23]:
new_data['Score'] =new_data['Score'].replace({1:0,2:0,4:1,5:1})
new_data['Score']

C:\Users\Student\AppData\Local\Temp\ipykernel_4928\193101030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Score'] =new_data['Score'].replace({1:0,2:0,4:1,5:1})


0        1
1        0
2        1
3        0
4        1
        ..
29997    1
29998    1
29999    1
30000    1
30001    1
Name: Score, Length: 30000, dtype: int64

In [24]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)                  # remove HTML
    text = re.sub(r'[^\w\s]', ' ', text)                # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()            # collapse spaces
    return text

new_data['clean_text'] = new_data['Text'].astype(str).apply(clean_text)


C:\Users\Student\AppData\Local\Temp\ipykernel_4928\834389873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['clean_text'] = new_data['Text'].astype(str).apply(clean_text)


In [25]:
new_data = new_data.loc[new_data['Score'] != 3].reset_index(drop=True)


In [26]:
X=new_data[['Score']].values
y=new_data['clean_text'].values

In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer 

maxlen=100
MAX_WORDS = 10000
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(new_data['clean_text'])  # Fit only once!
X_seq = tokenizer.texts_to_sequences(new_data['clean_text'])
X_pad = pad_sequences(X_seq, maxlen=maxlen, padding='post')



In [48]:

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)


In [49]:
# Use the existing tokenizer — no need to redefine it
X_seq = tokenizer.texts_to_sequences(new_data['clean_text'])
X_pad = pad_sequences(X_seq, maxlen=maxlen, padding='post')


In [50]:
model = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=16, input_length=maxlen),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


c:\Users\Student\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint('text_model.keras', save_best_only=True, monitor='val_loss')
    ]
)


Epoch 1/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 34s 14ms/step - accuracy: 0.8303 - loss: 0.4681 - val_accuracy: 0.8442 - val_loss: 0.3677
Epoch 2/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8577 - loss: 0.3265 - val_accuracy: 0.8868 - val_loss: 0.2596
Epoch 3/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9132 - loss: 0.2227 - val_accuracy: 0.9104 - val_loss: 0.2248
Epoch 4/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9298 - loss: 0.1819 - val_accuracy: 0.9031 - val_loss: 0.2291
Epoch 5/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9356 - loss: 0.1682 - val_accuracy: 0.9119 - val_loss: 0.2185
Epoch 6/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9457 - loss: 0.1462 - val_accuracy: 0.9129 - val_loss: 0.2194
Epoch 7/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9533 - loss: 0.1291 - val_accuracy: 0.9180 - val_loss: 0.2149
Epoch 8/10
688/688 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9554 - loss: 0.1217 - val_accuracy: 

In [54]:
import gradio as gr

def predict_review(review):
    review_seq = tokenizer.texts_to_sequences([review])
    review_pad = pad_sequences(review_seq, maxlen=maxlen, padding='post')
    prediction = model.predict(review_pad)
    return "Positive" if prediction[0][0] > 0.8 else "Negative"
iface = gr.Interface(
    fn=predict_review,
    inputs=gr.Textbox(lines=2, placeholder="Enter your review here..."),
    outputs="text",
    title="Review Sentiment Analysis",
    description="Enter a review to get its sentiment (Positive/Negative)."
)
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://5aad6c2e9ea7b023b1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
